# 1) Cargar la data

In [20]:
import os, pickle
import pandas as pd
from tabulate import tabulate
from rich import print

In [21]:
def leer_nombre_ficheros(elementos_SQ, temporalidades):
    """
    Encuentra los elementos anteriores al primer match entre la lista de elementos_SQ y temporalidades.

    Args:
        elementos_SQ (list): Lista de elementos a verificar.
        temporalidades (list): Lista de temporalidades a buscar.

    Returns:
        list: Lista de listas que contienen activos encontrados dentro del elemento en elementos_SQ.
        list: Lista de listas que contienen las temporalidades encontradas en cada elemento de elementos_SQ.
    """
    id_busqueda = []
    activos_encontrados = []
    temporalidades_encontradas = []
    
    for elemento in elementos_SQ:
        elementos_previos = []
        primer_elemento = elemento.split('_')[0]
        
        for temporalidad in temporalidades:
            index = elemento.find(temporalidad)
            if index != -1:
                elementos_previos.extend(elemento[:index].split('_')[1:-1])
                temporalidades_encontradas.append(elemento[index:].split('_')[:-1])
                break
        
        id_busqueda.append(primer_elemento)
        activos_encontrados.append(elementos_previos)
    
    return id_busqueda, activos_encontrados, temporalidades_encontradas

# DECORADOR
def styler(func):
    def wrapper(*args, **kwargs):
        df = func(*args, **kwargs)
        # formato para la impresion con estilo
        keys= list(df.columns)
        values = ['{:.2f}' for i in range(len(keys))]
        format_dict = dict(zip(keys, values))
        return df.style.format(format_dict).background_gradient(cmap='PRGn')
    return wrapper

# SE UTILIZA PARA VISUALIZAR LOS DATOS DEL OBJETO correlaciones

@styler
def get_correlation(data: dict, activo: str):
    """
    Devuelve la matriz de correlaciones para un activo determinado.

    Args:
        data (dict): Diccionario con los datos de correlaciones de todos los activos.
        activo (str): Activo a buscar en el diccionario.

    Returns:
        DataFrame: Matriz de correlaciones para el activo.

    ACLARACION: El DataFrame es tipo Styler. Por lo que algunas funcionalidades propias de los DataFrame se pierden.
    """
    # Filter the data dictionary based on the activo
    filtered_data = {
        key: values[activo.upper()]
        for key, values in data.items()
    }
    # Create the DataFrame
    df = pd.DataFrame(filtered_data)
    df.columns.name = 'correlacion {}'.format(activo)
    return df

def crear_menu(id_search, activos, temporalidades):
    menu = ""

    for i in range(len(id_search)):
        assets = ' - '.join(activos[i])
        temps = ' '.join(temporalidades[i])
        menu_item = f'{id_search[i]}) {assets} : {temps}\n'
        menu += menu_item

    return menu


In [23]:
# PROGRAMA PRINCIPAL
# Load the data
date_path = os.path.join(os.getcwd(), 'data')

# Get the date directory
directorios = os.listdir(date_path)
print(f' Directorios guardados: {directorios}')

directorios_elegido = directorios[-1]#TODO: acá va el input para cargar los datos correspondientes
file_dir = os.path.join(date_path, directorios_elegido)#TODO: estamos eligiendo el ultimo directorio, HARDCODED es la carpeta del dia de hoy
print(f' Directorio elegido: {directorios_elegido}')
#TODO: APLICAR crear_menu para elegir el directorio a cargar
print(f' Cantidad de solicitudes guardadas: {len(os.listdir(file_dir))//4}')

# obtener los nombres de los activos descargados
environment_temporalities = ['1d','4h','2h','1h','30m','15m','5m','3m', '1m']
elementos_SQ = [file for file in os.listdir(file_dir) if file.endswith('SQ.pickle')]

id_search, activos_encontrados, temporalidades = leer_nombre_ficheros(elementos_SQ, environment_temporalities)


if __name__ == '__main__':
    #TODO: crear una funcion main() que tenga argumentos de entrada, la fecha y luego los activos disponibles
    print(crear_menu(id_search, activos_encontrados, temporalidades))

    id_value = 2
    id_elegido = id_search[id_value]#TODO: acá va el input para cargar los datos correspondientes
    print(f' ID elegido: {id_elegido}')
    activos_elegidos = activos_encontrados[id_value]
    temporalidades_correspondientes = temporalidades[id_value]

    print(f' Activos elegidos: {activos_elegidos}')
    print(f' Temporalidades correspondientes: {temporalidades_correspondientes}')

    archivos_necesarios = ['.pickle','_SQ.pickle', '_CORR.pickle', '_TW.pickle']
    file_name_without_extension = f'{id_elegido}_' + '_'.join(activos_elegidos)+'_'+'_'.join(temporalidades_correspondientes)
    loaded_files = []
    for extension in archivos_necesarios:
        file_name = file_name_without_extension + extension
        file_path = os.path.join(file_dir, file_name)
        with open(file_path, 'rb') as file:
            loaded_files.append(pickle.load(file))
    data = {
        'data_raw': loaded_files[0],
        'data_SQ': loaded_files[1],
        'SQ_CORR': loaded_files[2],
        'time_window': loaded_files[3]
    }

Directorios guardados: ['04_junio_2023', '05_junio_2023', '06_junio_2023', '07_junio_2023', '08_junio_2023', 
'09_junio_2023', '10_junio_2023']

Directorio elegido: 10_junio_2023

Cantidad de solicitudes guardadas: 3

1) BTC - XRP - ETH : 4h 2h 1h 30m 15m 5m 3m 1m
2) BTC - XRP - ETH : 4h 2h 1h 30m 15m 5m 3m 1m
3) BTC - XRP - ETH : 15m 5m 3m 1m

ID elegido: 3

Activos elegidos: ['BTC', 'XRP', 'ETH']

Temporalidades correspondientes: ['15m', '5m', '3m', '1m']

# 2) Mostrar las correlaciones

Podría utilizarse la libreria Rich para armar tablas y exportarlas en formato txt


In [24]:
if __name__ == '__main__':
    asset = 'eth'

    df = get_correlation(data['SQ_CORR'], asset)
    display(df)

correlacion eth,9,12,14,18,20,22,30,40,50,60,70,80,90
15m,0.18,0.28,-0.19,0.31,0.34,0.36,0.51,0.84,0.94,0.68,0.72,0.76,0.79
5m,0.59,0.67,0.58,-0.40,-0.39,-0.38,-0.30,0.24,0.30,0.44,0.48,0.53,0.60
3m,0.25,0.38,0.52,0.44,0.49,0.55,0.60,-0.61,-0.50,0.23,0.26,0.29,0.32
1m,0.56,0.36,0.13,0.09,0.10,0.13,0.26,0.46,0.53,0.25,0.29,0.28,0.34


In [ ]:
# CREAR TABLA DE CORRELACIONES PARA EXPORTAR FORMATO TXT
# TODO: Obtener df en una variable tipo DataFrame, actualmente df es tipo Styler
## tabla para exportar a txt -> NO BORRAR!!
## tabla = tabulate(df, headers='keys', tablefmt='fancy_grid')
## print(f'Correlación {asset} con Squeeze Momentum Indicator')
## print(tabla)

In [9]:
# AUXILIAR PARA VER LA FLUCTUACION DE LAS MEDICIONES
# TODO: Manualmente leer dos archivos y ejecutarlos en celdas diferentes
if __name__ == '__main__':
    asset = 'eth'

    df = get_correlation(data['SQ_CORR'], asset)
    display(df)

correlacion eth,9,12,14,18,20,22,30,40,50,60,70,80,90
4h,0.59,0.52,0.63,0.48,0.48,0.47,0.37,0.28,0.21,0.56,0.35,0.07,0.01
2h,0.85,0.81,0.63,0.61,0.60,0.58,0.54,0.47,0.46,0.42,0.43,0.49,0.55
1h,0.77,0.94,0.90,0.84,0.83,0.82,0.74,0.61,0.57,0.49,0.50,0.51,0.51
30m,-0.01,0.17,0.53,0.83,0.89,0.93,0.95,0.83,0.79,0.61,0.59,0.56,0.53
15m,-0.50,-0.32,-0.06,0.34,0.37,0.41,0.62,0.90,0.94,0.66,0.70,0.74,0.78
5m,0.55,0.51,-0.02,-0.28,-0.27,-0.25,-0.15,0.27,0.33,0.46,0.52,0.61,0.71
3m,0.55,0.48,0.62,0.08,0.04,-0.00,-0.15,-0.31,-0.25,0.28,0.30,0.32,0.34
1m,0.59,0.57,0.53,0.34,0.34,0.37,0.39,0.58,0.58,-0.41,-0.34,-0.25,-0.03


# 3) Crear un registro de las notas de las lecturas

In [ ]:
# Revisar tradegestor DEMO

# 1) La temporalidad mas correlacionada
# La temporalidad mas correlacionada es 1h, excepto por len 40 y 50

# 2) Las temporalidades y los len que se van a analizar

# 2.1) 15 min en len 85
# 2.2) len 22 es fuerte en 1h, 30m, 15
# 2.3) len 30 5m


# 3) La temporalidad y los len que se van a visualizar
#4h len 12, 22
#1h len 80
#1h len 30
#30m len 14
#15m len 30
#5m len 30

# 4) Si se va a tomar una decisión dentro del mercado al respecto

# 5) Comentarios que se quieran hacer respecto al DI y al ADX

# 4) Ajustes de los datos previos a la visualizacion

En este caso en particular que estamos visualizando el gráfico de velas junto con el valor del squeeze momentum indicador, se opta por colocar la EMA de 200 periodos como si representara el punto cero del oscilador.

Como estamos trabajando con la variable data['data_SQ'][len][asset][temp], las variables que difieren una de otra son las relacionadas al squeeze momentum indicator, pero el precio de cierre de la serie es siempre el mismo, por lo tanto es correspondiente hacer los cálculo de las medias móviles solo a la serie de cierre precios para luego añadirsela a cada serie con su respectivo len.

len: longitud del periodo tomado para el cálculo del oscilador de momentum
asset: activo
temp: temporalidad

In [25]:
from screener.medias_moviles import get_moving_averages

df = data['data_raw']['ETH']['1h']

get_moving_averages(df, 'close', 200)


KeyError: '1h'

# 4) Visualización de la data

In [26]:
# Una forma auxiliar de imprimir dataframes
# La previsualización de los datos asociados a los axis tienen que ser visualizados de esta manera

import plotly.graph_objects as go
from rich import print
from datetime import datetime
from screener.medias_moviles import get_moving_averages
#print(f'Correlación {asset} con Squeeze Momentum Indicator')
#print(df.loc[:,70:])
#print(df.loc[:,30:60])
#print(df.loc[:,9:22])


In [27]:
def create_multiple_cs_plot(data, asset, temp, periods, add_mvg_avg=False, mvg_avg_period=200):
    """Función que se utiliza para crear multiples graficos en función de una temporalidad y varios periodos tomados de momentum (len)
    
    [description]

    [args]
    data (dict): diccionario con los datos de los activos
    asset (str): nombre del activo
    temp (str): temporalidad
    periods (list): lista de periodos a graficar

    [return]
    None
    """
    for period in periods:
        df = data['data_SQ'][period][asset][temp]
        try:
            period_window = data['time_window'][temp][period]
        except:
            period_window = df.index[0]
        candlestick_squeeze_plot(df, asset, temp, period, period_window, add_mvg_avg=add_mvg_avg, mvg_avg_period=mvg_avg_period)

def candlestick_squeeze_plot(data, asset, temp, period, period_window, add_mvg_avg=False, mvg_avg_period=200):
    """
    Funcion que crea un grafico de velas con un squeeze plot en graficos separados

    """


    features = ['open', 'close', 'high', 'low', 'value']
    df = data.loc[period_window:,features]
    if add_mvg_avg:
        df_mvg_avg = get_moving_averages(data, 'close', mvg_avg_period)
        df_mvg_avg = df_mvg_avg.loc[period_window:, :]
        scatters_mvg_avg = []
        for col in df_mvg_avg.columns:
            scatters_mvg_avg.append(go.Scatter(x=df_mvg_avg.index, y=df_mvg_avg[col], name=col))


    dist_tick = len(df)//7
    candlestick = go.Candlestick(x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'])

    bar = go.Bar(x=df.index, y=df['value'], name='squeeze')

    # CREACION DE LAS FIGURES
    if add_mvg_avg:
        fig = go.Figure(data=[candlestick, *scatters_mvg_avg])
    else:
        fig = go.Figure(data=[candlestick])
    fig2 = go.Figure(data=[bar])


    fig.layout.xaxis.type = 'category'
    fig.layout.xaxis.rangeslider.visible = False
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = df.index[dist_tick//2::dist_tick],#TODO:considerar este slide para balancear los ticks del eje x
            ticktext = df.index[:-dist_tick//2:dist_tick],#TODO:considerar este slide para balancear los ticks del eje x
        ),
        autosize=False,
        width=1200,
        height=600,
        margin=dict(
            
            l=50,
            r=50,
            b=50,
            t=50,
            pad=4
        ),
        paper_bgcolor="LightSteelBlue",
        title=f'{asset} {temp} | len squeeze: {period}',
        showlegend=False
    )

    fig2.update_layout(
        autosize=False,
        width=1200,
        height=300,
        margin=dict(
            
            l=50,
            r=50,
            b=1,
            t=10,
            pad=4
        ),
        paper_bgcolor="LightSteelBlue",
    )

    fig.show()
    fig2.show() 


In [29]:
#4h len 12, 22

asset = 'ETH'
temp = '1m'
periods = [50, 30, 14]

create_multiple_cs_plot(data, asset, temp, periods, add_mvg_avg=True, mvg_avg_period=200)

In [17]:
asset = 'ETH'
temp = '1h'
periods = [80, 22,12]

create_multiple_cs_plot(data, asset, temp, periods,add_mvg_avg=True, mvg_avg_period=200)

In [18]:
asset = 'ETH'
temp = '30m'
periods = [18, 50]

create_multiple_cs_plot(data, asset, temp, periods, add_mvg_avg=True, mvg_avg_period=55)

In [19]:
asset = 'ETH'
temp = '5m'
periods = [20, 50]

create_multiple_cs_plot(data, asset, temp, periods, add_mvg_avg=True, mvg_avg_period=55)

# 5) Exportar tablas, registros y visualizaciones